When use xlm-roberta. Restart the runtime after installing

In [22]:
! pip install sentencepiece

In [23]:
!pip install transformers

In [24]:
import re
import random
import numpy as np
import pandas as pd

import spacy

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer
from transformers import AutoModel, AutoTokenizer

from sklearn.model_selection import train_test_split, StratifiedKFold

from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score

import logging
logging.basicConfig(level = logging.ERROR)




from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'
#model_name = 'xlm-roberta-base'

OneStopEnglishCorpus

In [27]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("onestop_english")

VikiWiki

In [28]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [29]:
# !wget https://www.dropbox.com/s/bmov3bgxv53lpu4/all_lang.csv?dl=0 -O all_lang.csv

In [30]:
# df = pd.read_csv('/content/all_lang.csv')
# df['lang'].unique()

In [31]:
# lang='fr'

# dataset = df[df['lang']==lang]
# dataset = dataset.dropna()

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [33]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mean tree height

In [34]:
en_nlp = spacy.load('en_core_web_sm')

In [35]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
  depths = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    doc = en_nlp(sentence)
    depth = [walk_tree(sent.root, 0) for sent in doc.sents]
    depths.extend(depth)
  return np.mean(depths)

In [36]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.columns = ['content', 'labels']

In [69]:
num_diff = df.labels.unique()
diff = ['beginner','intermediate','advanced']

label = dict(zip(num_diff, diff))

diff = list(diff)

In [70]:
df['Difficulty'] = df.labels.replace(label)
df.head()

,content,labels,mean_tree_height,mean_sentence_len,avg_nounphrases,Difficulty
0,"When you see the word Amazon, what’s the first...",0,6.190476,25.400000,6.909091,beginner
1,"To tourists, Amsterdam still seems very libera...",0,5.727273,24.666667,6.869565,beginner
2,"Anitta, a music star from Brazil, has millions...",0,5.384615,20.538462,6.148148,beginner
3,Google has made maps of the world’s highest mo...,0,5.826087,23.260870,7.166667,beginner
4,The auction of a Banksy painting that disappea...,0,6.047619,24.526316,7.363636,beginner


In [37]:
df

,content,labels
0,"When you see the word Amazon, what’s the first...",0
1,"To tourists, Amsterdam still seems very libera...",0
2,"Anitta, a music star from Brazil, has millions...",0
3,Google has made maps of the world’s highest mo...,0
4,The auction of a Banksy painting that disappea...,0
...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2
563,Thousands of people protested on Australia’s b...,2
564,1 Race engineer \nA race engineer liaises betw...,2
565,More than one million British workers might be...,2


In [38]:
df['content'].iloc[0]

"When you see the word Amazon, what’s the first thing you think of – the world’s biggest forest, the longest river or the largest internet shop – and which do you think is most important? \nThese are the questions in a debate about the internet. Brazil and Peru have made objections to a bid made by the US online shop for the domain name, “.amazon”. \nAmazon has asked for its company name to be a top-level domain name (currently “.com”), but the South American governments say this would stop the use of this internet address for environmental protection, indigenous rights and other public interest uses. \nThere are many other disputed claims to names, including “.patagonia”. \nUntil now, the differences between commercial, governmental and other types of identity were easy to see in every internet address by the use of “.com”, “.gov” and 20 other categories. \nBut soon there are going to be more of these categories – or generic top-level domains (gTLDs) as they are technically known. \nT

In [39]:
get_mean_tree_height(df['content'].iloc[0])

6.190476190476191

In [40]:
get_mean_tree_height(df['content'].iloc[471])

6.666666666666667

In [41]:
df['mean_tree_height'] = df['content'].progress_apply(get_mean_tree_height)

  0%|          | 0/567 [00:00<?, ?it/s]

mean sentence length

In [42]:
sen_len = []
for i in df['content']:
  curr_sen_len = []
  for j in sent_tokenize(i):
    curr_sen_len.append(len(word_tokenize(j)))
  mean_len = np.mean(curr_sen_len)
  sen_len.append(mean_len)

In [43]:
sen_len[400:410]

[20.084745762711865,
 24.56756756756757,
 30.625,
 33.911764705882355,
 27.363636363636363,
 32.25,
 19.796610169491526,
 24.755102040816325,
 31.083333333333332,
 34.142857142857146]

In [44]:
sen_len[100:110]

[23.06896551724138,
 21.84375,
 21.75,
 20.625,
 22.785714285714285,
 20.12121212121212,
 24.304347826086957,
 18.32608695652174,
 19.9,
 23.296296296296298]

In [45]:
df['mean_sentence_len'] = sen_len

In [46]:
df

,content,labels,mean_tree_height,mean_sentence_len
0,"When you see the word Amazon, what’s the first...",0,6.190476,25.400000
1,"To tourists, Amsterdam still seems very libera...",0,5.727273,24.666667
2,"Anitta, a music star from Brazil, has millions...",0,5.384615,20.538462
3,Google has made maps of the world’s highest mo...,0,5.826087,23.260870
4,The auction of a Banksy painting that disappea...,0,6.047619,24.526316
...,...,...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2,4.771429,22.833333
563,Thousands of people protested on Australia’s b...,2,5.931034,28.653846
564,1 Race engineer \nA race engineer liaises betw...,2,6.023810,24.707317
565,More than one million British workers might be...,2,7.653846,39.260870


In [47]:
!pip install lingfeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 47.6 MB/s eta 0:00:00
  Created wheel for lingfeat: filename=lingfeat-1.0.0b19-py3-none-any.whl size=71127048 sha256=a9f6d2a7e22a856032b2c760a893c7aebd9aaca0200ff01b9f78339abcb9ab61
  Stored in directory: /root/.cache/pip/wheels/f0/6c/25/7cc4b4dbfc7c605bad9f8824afef1b4abc8ed18232972c86a8
Successfully built lingfeat


In [48]:
from lingfeat import extractor

Downloading: https://github.com/yzhangcs/parser/releases/download/v1.1.0/ptb.crf.con.lstm.char.zip to /root/.cache/supar/ptb.crf.con.lstm.char.zip
100%|██████████| 334M/334M [00:06<00:00, 50.8MB/s]


In [49]:
def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  return avg

In [50]:
df['avg_nounphrases'] = df['content'].progress_apply(get_avg_nounphrase)

  0%|          | 0/567 [00:00<?, ?it/s]

Dataset

In [51]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, tokenizer, max_len, feature_1=None, feature_2=None, feature_3=None):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer

        self.feature_1 = feature_1
        self.feature_2 = feature_2
        self.feature_3 = feature_3

        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]

        feature_1 = self.feature_1[item]
        feature_2 = self.feature_2[item]
        feature_3 = self.feature_3[item]

        # Tokenizing the texts, while also including special tokens
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long),
          'feature_1': torch.tensor([feature_1], dtype=torch.long),
          'feature_2': torch.tensor([feature_2], dtype=torch.long),
          'feature_3': torch.tensor([feature_3], dtype=torch.long)
        }


In [52]:
# df_temp, df_test = train_test_split(dataset, test_size=0.2, random_state=42)

# print("size of test set : " + str(df_test.shape[0]))

In [53]:
def generate_dataloader(df, tokenizer, max_len, batch_size):
    ds = DiffDataset(
        contents=df.content.to_numpy(),
        labels=df.labels.to_numpy(),

        feature_1 = df.mean_sentence_len.to_numpy(),
        feature_2 = df.mean_tree_height.to_numpy(),
        feature_3 = df.avg_nounphrases.to_numpy(),


        tokenizer=tokenizer,
        max_len=max_len,
      )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
      )


In [54]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [55]:
max_len = 128
batch_size = 8

# train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
# val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
# test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

# data = next(iter(train_dataloader))



In [56]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+3, n_classes)

    # Forward propagation function
    def forward(self, input_ids, attention_mask, feature_1, feature_2, feature_3):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]

        pooled_output = model_outs[1]
        pooled_output = torch.cat((pooled_output, feature_1, feature_2, feature_3) , dim=1)

        return self.out(pooled_output)


In [57]:
# len(dataset['labels'].unique())

In [58]:
# input_ids = data['input_ids'].to(device)
# attention_mask = data['attention_mask'].to(device)
# features = data['feature_1'].to(device)
# print(input_ids.shape)
# print(attention_mask.shape)
# print(features.shape)

In [59]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [60]:
def train(model, train_dataloader,optimizer,scheduler):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()


        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              feature_1 = d["feature_1"].to(device)
              feature_2 = d["feature_2"].to(device)
              feature_3 = d["feature_3"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, feature_1=feature_1, feature_2=feature_2, feature_3=feature_3)

              _, preds = torch.max(outputs, dim=1)

              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)

              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [61]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            feature_1 = d["feature_1"].to(device)
            feature_2 = d["feature_2"].to(device)
            feature_3 = d["feature_3"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                feature_1=feature_1,
                feature_2=feature_2,
                feature_3=feature_3
              )

            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives)
            # for each class and label, and find their average weighted by support
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts

In [62]:
def testing(model, dataloader):
  model = model.eval()

  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      feature_1 = d["feature_1"].to(device)
      feature_2 = d["feature_2"].to(device)
      feature_3 = d["feature_3"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        feature_1=feature_1,
        feature_2=feature_2,
        feature_3=feature_3
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [63]:
best_model_path = "./bert_model_baseline.bin"
def save_best_model(model):
  torch.save(model.state_dict(), best_model_path)

def load_best_model(model):
  model.load_state_dict(torch.load(best_model_path))
  return model

In [64]:
class EarlyStopping:
    def __init__(self, model, patience=1):
        self.patience = patience  # number of times to allow for no improvement before stopping the execution
        self.counter = 0  # count the number of times the validation accuracy not improving
        self.min_validation_loss = np.inf
        self.model = model
        self.max_acc = 0

    # return True when validation loss is not decreased by the `min_delta` for `patience` times
    def early_stop_check(self, val_acc):
        if (val_acc > self.max_acc):
            self.max_acc = val_acc
            self.counter = 0  # reset the counter if validation loss decreased at least by min_delta
            save_best_model(model)
        else:
            self.counter += 1 # increase the counter if validation loss is not decreased by the min_delta
            if self.counter >= self.patience:
                return True
        return False


In [72]:
df

,content,labels,mean_tree_height,mean_sentence_len,avg_nounphrases,Difficulty
0,"When you see the word Amazon, what’s the first...",0,6.190476,25.400000,6.909091,beginner
1,"To tourists, Amsterdam still seems very libera...",0,5.727273,24.666667,6.869565,beginner
2,"Anitta, a music star from Brazil, has millions...",0,5.384615,20.538462,6.148148,beginner
3,Google has made maps of the world’s highest mo...,0,5.826087,23.260870,7.166667,beginner
4,The auction of a Banksy painting that disappea...,0,6.047619,24.526316,7.363636,beginner
...,...,...,...,...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2,4.771429,22.833333,6.333333,advanced
563,Thousands of people protested on Australia’s b...,2,5.931034,28.653846,8.666667,advanced
564,1 Race engineer \nA race engineer liaises betw...,2,6.023810,24.707317,7.571429,advanced
565,More than one million British workers might be...,2,7.653846,39.260870,11.214286,advanced


In [73]:
b_accuracy = 0
epochs = 10
n_fold = 10
random_state = 42
folds_acc = []
histories = []


k_fold = StratifiedKFold(n_splits=n_fold, random_state=random_state, shuffle=True)
y = df.drop(["content", "Difficulty", "mean_tree_height", "mean_sentence_len", "avg_nounphrases"], axis=1)

for i, (temp_index, test_index) in enumerate(k_fold.split(df, y)):
    print(" ------   Fold {}  ------- ".format(i+1), end="\n")
    df_temp = df.iloc[temp_index]
    df_train, df_val = train_test_split(df_temp, test_size=0.2, random_state=42)
    df_test = df.iloc[test_index]
    train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
    val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
    test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)



    data = next(iter(train_dataloader))
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    feature_1 = data['feature_1'].to(device)
    feature_2 = data['feature_2'].to(device)

    model = DiffClassifier(len(diff))
    # Running the classifier on GPU
    model = model.to(device)

    optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  weight_decay = 0.2,
                  correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0.1,
                                                num_training_steps=len(train_dataloader)*epochs)
    loss_fn = nn.CrossEntropyLoss().to(device)

    history = defaultdict(list)

    early_stopping = EarlyStopping(model, patience=4)

    for epoch in range(epochs):

      print(f'Epoch {epoch + 1}/{epochs}')
      print('-' * 20)

      train_acc, train_loss, train_preci = train(
            model,
            train_dataloader,
            optimizer,
            scheduler,
          )
      print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['train_precision'].append(train_preci)

      val_acc, val_loss, val_preci = eval(
            model,
            val_dataloader,
            loss_fn,
            device,
            len(df_val),
          )

      print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')
      print()

      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)
      history['val_precision'].append(val_preci)

      histories.append(history)

      if early_stopping.early_stop_check(val_acc):
          break

      # if val_acc > b_accuracy:
      #   torch.save(model.state_dict(), best_model_path)
      #   b_accuracy = val_acc

    model = load_best_model(model)
    y_contents, y_pred, y_pred_probs, y_test = testing(
          model,
          test_dataloader
        )
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy for fold {}: {:.2f}%\n".format(i+1, acc*100))
    folds_acc.append(acc)

 ------   Fold 1  ------- 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/10
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Train : Loss 0.00034724919618490865, Accuracy : 100.00 %, Precision : 1.0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.7774724897082286, Accuracy : 87.38 %, Precision : 0.9282509157509158



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Test Accuracy for fold 10: 91.07%



In [74]:
print("The accuracy score is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score is: 91.698%


In [ ]:
model.load_state_dict(torch.load("best_bert_features.bin"))

y_contents, y_pred, y_pred_probs, y_test = testing(
  model,
  test_dataloader
)

out_df = pd.DataFrame()
out_df['content'] = y_contents
out_df['true_label'] = y_test
out_df['pred_label'] = y_pred

In [ ]:
out_df